# Домашнее задание № 4 часть 1 (курс МТС, команда 33)

## Подготовка окружения для проведения эксперимента

In [1]:
# устанавливаем необходимые зависимости
!pip install rectools > None
!pip install optuna > None

In [2]:
# импортируем зависимости

import datetime
import optuna
import dill
import nmslib
import pandas as pd
import numpy as np

from implicit.als import AlternatingLeastSquares
from rectools.metrics import Precision, Recall, MAP, calc_metrics
from rectools.models import ImplicitALSWrapperModel, LightFMWrapperModel
from rectools import Columns
from rectools.dataset import Dataset
from lightfm import LightFM
from lightfm.data import Dataset as LFMDataset

Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [3]:
# скачиваем датасеты

!wget https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip

--2022-12-12 05:33:12--  https://storage.yandexcloud.net/itmo-recsys-public-data/kion_train.zip
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78795385 (75M) [application/zip]
Saving to: ‘kion_train.zip’

kion_train.zip      100%[===================>]  75.14M  21.1MB/s    in 4.4s    

2022-12-12 05:33:18 (17.0 MB/s) - ‘kion_train.zip’ saved [78795385/78795385]



In [4]:
!ls

None  __notebook__.ipynb  kion_train.zip


In [5]:
!unzip ./kion_train.zip 

Archive:  ./kion_train.zip
   creating: kion_train/
  inflating: kion_train/interactions.csv  
  inflating: __MACOSX/kion_train/._interactions.csv  
  inflating: kion_train/users.csv    
  inflating: __MACOSX/kion_train/._users.csv  
  inflating: kion_train/items.csv    
  inflating: __MACOSX/kion_train/._items.csv  


In [6]:
N = 10
METRICS = {"map@10": MAP(k=10),"recall@10": Recall(k=10)}

## Подготовка датасета

In [7]:
def get_datasets():
    inter_df = pd.read_csv('kion_train/interactions.csv')
    user_df = pd.read_csv('kion_train/users.csv')
    item_df = pd.read_csv('kion_train/items.csv')
    inter_df.rename(columns={'last_watch_dt': 'datetime','total_dur': 'weight',},inplace=True)
    inter_df['datetime'] = pd.to_datetime(inter_df['datetime'])
    inter_df[Columns.Weight] = np.where(inter_df['watched_pct'] > 10, 3, 1)
    return inter_df, user_df, item_df

In [8]:
inter_df, user_df, item_df = get_datasets()

In [9]:
# визуализируем датасеты

inter_df.head(10)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,3,72.0
1,699317,1659,2021-05-29,3,100.0
2,656683,7107,2021-05-09,1,0.0
3,864613,7638,2021-07-05,3,100.0
4,964868,9506,2021-04-30,3,100.0
5,1032142,6686,2021-05-13,3,100.0
6,1016458,354,2021-08-14,3,25.0
7,884009,693,2021-08-04,3,14.0
8,648682,1449,2021-06-13,3,75.0
9,203219,13582,2021-08-22,3,100.0


In [10]:
item_df.head(10)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."
5,854,film,Северо-Юг,NaN,2015.0,"драмы, русские",Россия,NaN,16.0,NaN,Юрий Грубник,"Алексей Воронин, Алексей Мурашов, Алена Алинин...",Авторский фильм молодого режиссера Юрия Грубни...,"Северо-Юг, 2015, Россия, бандиты, гангстеры, б..."
6,1468,film,Марья-искусница,NaN,1960.0,"фильмы, сказки, приключения, советские, семейн...",СССР,NaN,6.0,NaN,Александр Роу,"Александр Баранов, Александр Хвыля, Анатолий К...",Два прославленных советских сказочника – писат...,"Марья-искусница, 1960, СССР, преодоление, труд..."
7,11114,film,Принцесса Лебедь: Пират или принцесса,"The Swan Princess: Princess Tomorrow, Pirate T...",2016.0,"для детей, сказки, полнометражные, зарубежные,...",США,NaN,6.0,Sony Pictures,Ричард Рич,"Брайан Ниссен, Гарднер Джаэс, Грант Дураззо, Д...",Анимационная сказка о непоседливой принцессе Э...,"Принцесса, Лебедь, Пират, или, принцесса, 2016..."
8,9853,film,Лабиринты прошлого,Todos lo saben,2018.0,"криминал, детективы, драмы, зарубежные, триллеры",Испания,NaN,16.0,NaN,Асгар Фархади,"Барбара Ленни, Инма Куэста, Карла Кампра, Пене...","Испанка Лаура, давно переехавшая в Буэнос-Айре...","Лабиринты, прошлого, 2018, Испания, семейные, ..."
9,8604,film,Третья попытка,NaN,2013.0,"русские, мелодрамы",Россия,NaN,12.0,NaN,Игорь Мужжухин,"Александр Асташенок, Александр Пашков, Андрей ...","Екатерина Рябова, Александр Асташенок и Алекса...","Третья, попытка, 2013, Россия, любовь, измена,..."


In [11]:
user_df.head(10)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0
6,391756,age_25_34,income_0_20,М,0
7,15878,age_25_34,income_40_60,М,1
8,846063,age_35_44,income_40_60,Ж,1
9,401219,age_35_44,income_40_60,Ж,0


In [12]:
# формируем трейновый и тестовый датасет

last_date = inter_df[Columns.Datetime].max().normalize()

train = inter_df[inter_df[Columns.Datetime] < last_date - pd.Timedelta(days=7)].copy()
test = inter_df[inter_df[Columns.Datetime] >= last_date - pd.Timedelta(days=7)].copy()

print(f"Размерность трейна : {train.shape}")
print(f"Размерность теста: {test.shape}")

Размерность трейна : (4985269, 5)
Размерность теста: (490982, 5)


In [13]:
# убираем холодных пользователей из теста

cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)

In [14]:
print(user_df.isna().sum())

user_id         0
age         14095
income      14776
sex         13831
kids_flg        0
dtype: int64


In [15]:
# заполняем пустые значения

user_df.fillna('Unknown', inplace=True)
user_df = user_df.loc[user_df[Columns.User].isin(train[Columns.User])].copy()
user_df.head()

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0
5,1037719,age_45_54,income_60_90,М,0


In [16]:
# извлекаем возраст, пол и доход пользователей

user_features_frames = []
for feature in ["age", "sex", "income"]:
    feature_frame = user_df.reindex(columns=[Columns.User, feature])
    feature_frame.columns = ["id", "value"]
    feature_frame["feature"] = feature
    user_features_frames.append(feature_frame)
user_features = pd.concat(user_features_frames)
user_features.head()

,id,value,feature
0,973171,age_25_34,age
1,962099,age_18_24,age
3,721985,age_45_54,age
4,704055,age_35_44,age
5,1037719,age_45_54,age


In [17]:
item_df = item_df.loc[item_df[Columns.Item].isin(train[Columns.Item])].copy()
item_df.head()

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [18]:
# определяем тип контента

print(item_df.content_type.unique())

['film' 'series']


In [19]:
# извлекаем жанры для каждого фильма и сериала

item_df["genre"] = item_df["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = item_df[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
genre_feature.head()

,id,value,feature
0,10711,драмы,genre
0,10711,зарубежные,genre
0,10711,детективы,genre
0,10711,мелодрамы,genre
1,2508,зарубежные,genre


In [20]:
# извлекаем тип контента для каждого айтема

content_feature = item_df.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
content_feature.head()

,id,value,feature
0,10711,film,content_type
1,2508,film,content_type
2,10716,film,content_type
3,7868,film,content_type
4,16268,film,content_type


In [21]:
# извлекаем квантиль года выпуска для каждого айтема

item_df['binned_r_year'] = pd.qcut(item_df['release_year'], q=10, labels=list(range(10)))
release_year_feature = item_df.reindex(columns=[Columns.Item, "binned_r_year"])
release_year_feature.columns = ["id", "value"]
release_year_feature["feature"] = "binned_r_year"
release_year_feature.head()

,id,value,feature
0,10711,1,binned_r_year
1,2508,4,binned_r_year
2,10716,3,binned_r_year
3,7868,5,binned_r_year
4,16268,0,binned_r_year


In [22]:
# извлекаем местоположение студии для каждого айтема

item_df["country"] = item_df["countries"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
country_feature = item_df[["item_id", "country"]].explode("country")
country_feature.columns = ["id", "value"]
country_feature["feature"] = "country"
country_feature.head()

,id,value,feature
0,10711,испания,country
1,2508,сша,country
2,10716,канада,country
3,7868,великобритания,country
4,16268,ссср,country


In [23]:
# конкатенируем таблицы с фичами и проверяем по рандомному id

item_features = pd.concat((genre_feature, content_feature, country_feature, release_year_feature))
item_features[item_features['id'] == 222]

,id,value,feature
4326,222,драмы,genre
4326,222,русские,genre
4326,222,film,content_type
4326,222,ссср,country
4326,222,0,binned_r_year


In [24]:
# создаем датасет с фичами

DATASET = Dataset.construct(
    interactions_df=train,
    user_features_df=user_features,
    cat_user_features=["age", "sex", "income"],
    item_features_df=item_features,
    cat_item_features=["genre", "country", 'binned_r_year', 'content_type'],
)

In [25]:
DATASET

Dataset(user_id_map=IdMap(external_ids=array([176549, 699317, 656683, ..., 882138, 805174, 648596])), item_id_map=IdMap(external_ids=array([ 9506,  1659,  7107, ..., 13516, 13019, 10542])), interactions=Interactions(df=         user_id  item_id  weight   datetime
0              0        0     3.0 2021-05-11
1              1        1     3.0 2021-05-29
2              2        2     1.0 2021-05-09
3              3        3     3.0 2021-07-05
4              4        0     3.0 2021-04-30
...          ...      ...     ...        ...
5476244    69627      219     3.0 2021-08-02
5476245    40052      132     1.0 2021-05-12
5476246   896790      318     1.0 2021-08-13
5476247   206604     2546     3.0 2021-04-13
5476249     7236     1609     3.0 2021-04-19

[4985269 rows x 4 columns]), user_features=SparseFeatures(values=<896791x17 sparse matrix of type '<class 'numpy.float32'>'
	with 2091408 stored elements in Compressed Sparse Row format>, names=(('age', 'age_25_34'), ('age', 'age_18_24'), (

In [26]:
# определяем пользователей для теста

TEST_USERS = test[Columns.User].unique()

## Обучение факторизационных машин

### 1. ALS - Alternating Least Squares

In [27]:
def objective_train_als_model(trial):
    n_factors = trial.suggest_int("n_factors", low=32, high=128, step=32)
    regularization = trial.suggest_float("regularization", low=0.01, high=0.51, step=0.1)
    model = ImplicitALSWrapperModel(
      model=AlternatingLeastSquares(
        factors=n_factors, 
        regularization=regularization,
        random_state=100, 
      ),
      fit_features_together=True,
    )
    model.fit(DATASET)
    rec = model.recommend(
      users=TEST_USERS,
      dataset=DATASET,
      k=N,
      filter_viewed=True,
    )
    metrics = calc_metrics(METRICS, rec, test, train)
    return metrics['map@10'], metrics['recall@10']

def save_best_model(trial):
    n_factors = trial.suggest_int("n_factors", low=32, high=128, step=32)
    regularization = trial.suggest_float("regularization", low=0.01, high=0.51, step=0.1)
    model = ImplicitALSWrapperModel(
      model=AlternatingLeastSquares(
        factors=n_factors, 
        regularization=regularization,
        random_state=100, 
      ),
      fit_features_together=True,
    )
    model.fit(DATASET)
    with open('als_model.dill', 'wb') as f:
        dill.dump(model.model, f)

In [28]:
def print_train_loop_inf(train_loop):
    print("Завершенных сессий: ", len(train_loop.trials))
    print("Результаты: ")
    trials = train_loop.best_trials
    for trial in trials:
        print("  Параметры: ")
        for key, value in trial.params.items():
            print("    {}: {}".format(key, value))

In [29]:
%%time

# обучаем ALS модель

study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(objective_train_als_model, n_trials=10)

[I 2022-12-12 05:33:33,202] A new study created in memory with name: no-name-e21fb1fb-23b6-4c1e-ab6a-94d0fb1d9b09
/opt/conda/lib/python3.7/site-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
/opt/conda/lib/python3.7/site-packages/rectools/models/implicit_als.py:270: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 280 to 288 (increasing latent factors from 64 to 72)
  "GPU training requires number of factors to be a multiple of 32."
[I 2022-12-12 05:37:28,933] Trial 0 finished with values: [0.07932188677194683, 0.15315738030973283] and parameters: {'n_factors': 64, 'regularization': 0.41000000000000003}. 
/opt/conda/lib/python3.7/site-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting s

CPU times: user 58min 50s, sys: 9min 42s, total: 1h 8min 33s
Wall time: 40min 54s


In [30]:
print_train_loop_inf(study)

Завершенных сессий:  10
Результаты: 
  Параметры: 
    n_factors: 128
    regularization: 0.11
  Параметры: 
    n_factors: 96
    regularization: 0.51
  Параметры: 
    n_factors: 96
    regularization: 0.51
  Параметры: 
    n_factors: 128
    regularization: 0.31000000000000005


In [31]:
# визуализируем лучший вариант

study.best_trials

[FrozenTrial(number=3, values=[0.07950833475590335, 0.15308069793562015], datetime_start=datetime.datetime(2022, 12, 12, 5, 45, 11, 753506), datetime_complete=datetime.datetime(2022, 12, 12, 5, 49, 50, 980110), params={'n_factors': 128, 'regularization': 0.11}, distributions={'n_factors': IntDistribution(high=128, log=False, low=32, step=32), 'regularization': FloatDistribution(high=0.51, log=False, low=0.01, step=0.1)}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, trial_id=3, state=TrialState.COMPLETE, value=None),
 FrozenTrial(number=5, values=[0.07940032170591849, 0.15509207332340064], datetime_start=datetime.datetime(2022, 12, 12, 5, 53, 21, 598496), datetime_complete=datetime.datetime(2022, 12, 12, 5, 57, 40, 385166), params={'n_factors': 96, 'regularization': 0.51}, distributions={'n_factors': IntDistribution(high=128, log=False, low=32, step=32), 'regularization': FloatDistribution(high=0.51, log=False, low=0.01, step=0.1)}, user_attrs={}, system_

In [32]:
# сохраняем лучший вариант

save_best_model(study.best_trials[0])

/opt/conda/lib/python3.7/site-packages/rectools/dataset/features.py:399: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
/opt/conda/lib/python3.7/site-packages/rectools/models/implicit_als.py:270: UserWarning: GPU training requires number of factors to be a multiple of 32. Increasing factors from 344 to 352 (increasing latent factors from 128 to 136)
  "GPU training requires number of factors to be a multiple of 32."


### 2. Обучение LightFM

In [33]:
def objective_train_light_fm(trial):
    n_factors = trial.suggest_int("n_factors", low=32, high=128, step=32)
    loss = trial.suggest_categorical("loss", choices=['logistic', 'bpr', 'warp'])
    lr = trial.suggest_float("lr", low=0.05, high=0.25, step=0.05)
    item_alpha = trial.suggest_float("item_alpha", low=0.0, high=0.1, step=0.05)
    user_alpha = trial.suggest_float("item_alpha", low=0.0, high=0.1, step=0.05)
    model = LightFMWrapperModel(
      model=LightFM(
        no_components=n_factors, 
        loss=loss, 
        random_state=100,
        learning_rate=lr,
        user_alpha=user_alpha,
        item_alpha=item_alpha,
      ),
      epochs=2,
      num_threads=1,
    )
    model.fit(DATASET)
    rec = model.recommend(
      users=TEST_USERS,
      dataset=DATASET,
      k=N,
      filter_viewed=True,
    )
    metrics = calc_metrics(METRICS, rec, test, train)
    return metrics['map@10'], metrics['recall@10']

def save_best_model(trial):
    n_factors = trial.suggest_int("n_factors", low=32, high=128, step=32)
    loss = trial.suggest_categorical("loss", choices=['logistic', 'bpr', 'warp'])
    lr = trial.suggest_float("lr", low=0.05, high=0.25, step=0.05)
    item_alpha = trial.suggest_float("item_alpha", low=0.0, high=0.1, step=0.05)
    user_alpha = trial.suggest_float("item_alpha", low=0.0, high=0.1, step=0.05)
    model = LightFMWrapperModel(
      model=LightFM(
        no_components=n_factors, 
        loss=loss, 
        random_state=100,
        learning_rate=lr,
        user_alpha=user_alpha,
        item_alpha=item_alpha,
      ),
      epochs=3,
      num_threads=1,
    )
    model.fit(DATASET)
    with open('lightfm.dill', 'wb') as f:
        dill.dump(model.model, f)

In [34]:
%%time

# обучаем LightFM модель

study = optuna.create_study(directions=["maximize", "maximize"])
study.optimize(objective_train_light_fm, n_trials=4)

[I 2022-12-12 06:15:35,764] A new study created in memory with name: no-name-63898d84-1eed-421a-a600-2dedc036e8d5
[I 2022-12-12 06:19:00,370] Trial 0 finished with values: [1.129848947351972e-05, 3.447708556934683e-05] and parameters: {'n_factors': 32, 'loss': 'bpr', 'lr': 0.2, 'item_alpha': 0.1}. 
[I 2022-12-12 06:23:07,950] Trial 1 finished with values: [0.001115605084148824, 0.004406561311191432] and parameters: {'n_factors': 64, 'loss': 'warp', 'lr': 0.2, 'item_alpha': 0.0}. 
[I 2022-12-12 06:27:50,392] Trial 2 finished with values: [1.0371808594495474e-06, 8.29744687559638e-06] and parameters: {'n_factors': 64, 'loss': 'bpr', 'lr': 0.15000000000000002, 'item_alpha': 0.1}. 
[I 2022-12-12 06:35:29,510] Trial 3 finished with values: [0.0, 0.0] and parameters: {'n_factors': 128, 'loss': 'bpr', 'lr': 0.25, 'item_alpha': 0.05}. 


CPU times: user 24min 38s, sys: 4min 3s, total: 28min 42s
Wall time: 19min 53s


In [35]:
print_train_loop_inf(study)

Завершенных сессий:  4
Результаты: 
  Параметры: 
    n_factors: 64
    loss: warp
    lr: 0.2
    item_alpha: 0.0


In [36]:
study.best_trials

[FrozenTrial(number=1, values=[0.001115605084148824, 0.004406561311191432], datetime_start=datetime.datetime(2022, 12, 12, 6, 19, 0, 371567), datetime_complete=datetime.datetime(2022, 12, 12, 6, 23, 7, 950707), params={'n_factors': 64, 'loss': 'warp', 'lr': 0.2, 'item_alpha': 0.0}, distributions={'n_factors': IntDistribution(high=128, log=False, low=32, step=32), 'loss': CategoricalDistribution(choices=('logistic', 'bpr', 'warp')), 'lr': FloatDistribution(high=0.25, log=False, low=0.05, step=0.05), 'item_alpha': FloatDistribution(high=0.1, log=False, low=0.0, step=0.05)}, user_attrs={}, system_attrs={'nsga2:generation': 0}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)]

In [37]:
save_best_model(study.best_trials[0])

## Создание аватаров

Для выполнения данного задания мы решили создать 3 аватара: 1) молодой мужчина, интересующийся немецкими ужастиками; 2) пожилой мужчина, увлекающийся американскими боевиками; 3) женщина среднего возраста, предпочитающая французские комедии.

In [38]:
def create_watched_list(user_id, country, genre):
    data = item_df[(item_df['countries'].str.contains(country)) & item_df['genres'].str.contains(genre)].sample(7, random_state=100)['title'].values
    avatar_watched_list = pd.DataFrame({"user_id": user_id, "title": data})
    avatar_watched_list = avatar_watched_list.merge(item_df[["item_id", "title", "genres"]], on="title")
    return avatar_watched_list

def create_avatar(user_id, sex, age, income):
    avatar = pd.DataFrame([{'id': user_id, 'value': sex, 'feature': 'sex'},
                            {'id': user_id, 'value': age, 'feature': 'age'},
                            {'id': user_id, 'value': income, 'feature': 'income'},]
    )
    return avatar

In [39]:
ger_horror_watched = create_watched_list('ger_horror', 'Германия', 'ужасы')
ger_horror_watched

,user_id,title,item_id,genres
0,ger_horror,Голоса,11605,"ужасы, триллеры, криминал, комедии"
1,ger_horror,Голоса,779,"триллеры, короткометражные"
2,ger_horror,Мумия: Гробница Императора Драконов,14099,"боевики, ужасы, триллеры, фэнтези"
3,ger_horror,Свора,1256,"приключения, зарубежные, фантастика, триллеры,..."
4,ger_horror,Смертельный декабрь,5577,"зарубежные, ужасы"
5,ger_horror,"Дом, который построил Джек (жестовым языком)",9699,"драмы, ужасы, зарубежные, криминал"
6,ger_horror,Голем,9151,"триллеры, криминал, детективы"
7,ger_horror,Голем,587,"драмы, ужасы, фэнтези"
8,ger_horror,Охотники на ведьм,11680,"боевики, ужасы, фэнтези"


In [40]:
ger_horror = create_avatar('ger_horror', 'М', 'age_18_24', 'income_0_20')
ger_horror

,id,value,feature
0,ger_horror,М,sex
1,ger_horror,age_18_24,age
2,ger_horror,income_0_20,income


In [41]:
rambo_watched = create_watched_list('rambo', 'США', 'боевик')
rambo_watched

,user_id,title,item_id,genres
0,rambo,Разборка в Маниле,9601,боевики
1,rambo,Проповедник с пулемётом,12302,"боевики, драмы"
2,rambo,Стартрек 8: Первый контакт,6972,"боевики, фантастика, приключения"
3,rambo,Побег,10758,"криминал, приключения, драмы, триллеры, мелодр..."
4,rambo,Побег,16025,"драмы, короткометражные, приключения, комедии"
5,rambo,Звёздная принцесса и силы зла,11868,"мультсериалы, приключения, драмы, фантастика, ..."
6,rambo,Парный удар,10036,"приключения, драмы, зарубежные, боевики, комедии"
7,rambo,Джон Уик 3,10073,"боевики, триллеры, криминал"


In [42]:
rambo = create_avatar('rambo', 'М', 'age_65_inf', 'income_60_90')
rambo

,id,value,feature
0,rambo,М,sex
1,rambo,age_65_inf,age
2,rambo,income_60_90,income


In [43]:
madam_hihi_watched = create_watched_list('madam_hihi', 'Франция', 'комедии')
madam_hihi_watched

,user_id,title,item_id,genres
0,madam_hihi,Красивый бандит,14172,"драмы, детективы, комедии"
1,madam_hihi,Ребенок любой ценой,12508,комедии
2,madam_hihi,Твоя рука в моей руке,11341,"мелодрамы, зарубежные, комедии"
3,madam_hihi,Каникулы маленького Николя,3747,"семейное, комедии"
4,madam_hihi,Playmobil фильм: Через вселенные (с тифлокомме...,12450,"приключения, зарубежные, семейное, фэнтези, ко..."
5,madam_hihi,День выборов по-французски,8024,комедии
6,madam_hihi,Бернард,16356,"мультсериалы, семейное, комедии"
7,madam_hihi,Бернард,5829,"для детей, сериалы, про животных, зарубежные, ..."


In [44]:
madam_hihi = create_avatar('madam_hihi', 'Ж', 'age_35_44', 'income_90_150')
madam_hihi

,id,value,feature
0,madam_hihi,Ж,sex
1,madam_hihi,age_35_44,age
2,madam_hihi,income_90_150,income


In [45]:
avatar_train = pd.concat([train, ger_horror_watched.drop(['title', 'genres'], axis=1),
                          rambo_watched.drop(['title', 'genres'], axis=1), 
                          madam_hihi_watched.drop(['title', 'genres'], axis=1),], sort=False)
avatar_train.head(10)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,3.0,72.0
1,699317,1659,2021-05-29,3.0,100.0
2,656683,7107,2021-05-09,1.0,0.0
3,864613,7638,2021-07-05,3.0,100.0
4,964868,9506,2021-04-30,3.0,100.0
5,1032142,6686,2021-05-13,3.0,100.0
6,1016458,354,2021-08-14,3.0,25.0
7,884009,693,2021-08-04,3.0,14.0
8,648682,1449,2021-06-13,3.0,75.0
10,791466,8199,2021-07-27,1.0,9.0


In [46]:
avatar_train.tail(10)

,user_id,item_id,datetime,weight,watched_pct
6,rambo,10036,NaT,NaN,NaN
7,rambo,10073,NaT,NaN,NaN
0,madam_hihi,14172,NaT,NaN,NaN
1,madam_hihi,12508,NaT,NaN,NaN
2,madam_hihi,11341,NaT,NaN,NaN
3,madam_hihi,3747,NaT,NaN,NaN
4,madam_hihi,12450,NaT,NaN,NaN
5,madam_hihi,8024,NaT,NaN,NaN
6,madam_hihi,16356,NaT,NaN,NaN
7,madam_hihi,5829,NaT,NaN,NaN


In [47]:
avatar_train['datetime'] = avatar_train['datetime'].fillna(datetime.datetime(2021, 1, 1))
avatar_train['weight'] = avatar_train['weight'].fillna(3)
avatar_train['watched_pct'] = avatar_train['watched_pct'].fillna(80)

In [48]:
DATASET = LFMDataset()
DATASET.fit(users=avatar_train["user_id"].values,items=avatar_train["item_id"].values,)
inter_matrix, _ = DATASET.build_interactions(zip(*avatar_train[["user_id", "item_id"]].values.T))

In [49]:
model = LightFM(learning_rate=0.05, 
                loss='warp', 
                no_components=64,
                random_state=100,
)
model.fit(interactions = inter_matrix, 
                        epochs=2,
                        num_threads=20,
)

In [50]:
id_item_map = {v: k for k, v in DATASET._item_id_mapping.items()}

In [51]:
def get_recos(user_id, model, inter_matrix, user_to_id, id_to_item, n_recommendations = N):
    inner_id = user_to_id[user_id]
    scores = model.predict(user_ids=inner_id,
                            item_ids=np.arange(inter_matrix.shape[1]),
                            num_threads=20)
    watched_items = inter_matrix.col[inter_matrix.row == inner_id]
    scores[watched_items] = -np.inf
    reco_item_inner_ids = np.argpartition(scores, -np.arange(n_recommendations))[-n_recommendations:][::-1]
    recos = [id_to_item[x] for x in reco_item_inner_ids]
    return recos


In [52]:
# рекомендации для аватара ger_horror

user_id = "ger_horror"

recommended_items = get_recos(user_id=user_id,
                              model=model,
                              inter_matrix=inter_matrix,
                              user_to_id=DATASET._user_id_mapping,
                              id_to_item=id_item_map,
                              n_recommendations=N
)

ger_horror_recos = pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(item_df[["item_id", "title", "genres"]])
ger_horror_recos

,user_id,item_id,title,genres
0,ger_horror,10440,Хрустальный,"триллеры, детективы"
1,ger_horror,9728,Гнев человеческий,"боевики, триллеры"
2,ger_horror,15297,Клиника счастья,"драмы, мелодрамы"
3,ger_horror,2657,Подслушано,"драмы, триллеры"
4,ger_horror,14741,Цвет из иных миров,"фантастика, ужасы"
5,ger_horror,7571,100% волк,"мультфильм, приключения, семейное, фэнтези, ко..."
6,ger_horror,4151,Секреты семейной жизни,комедии
7,ger_horror,13865,Девятаев,"драмы, военные, приключения"
8,ger_horror,14431,Приворот. Чёрное венчание,"ужасы, триллеры, мелодрамы"
9,ger_horror,3734,Прабабушка легкого поведения,комедии


In [53]:
# рекомендации для аватара rambo

user_id = "rambo"

recommended_items = get_recos(user_id=user_id,
                              model=model,
                              inter_matrix=inter_matrix,
                              user_to_id=DATASET._user_id_mapping,
                              id_to_item=id_item_map,
                              n_recommendations=N
)

rambo_recos = pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(item_df[["item_id", "title", "genres"]])
rambo_recos

,user_id,item_id,title,genres
0,rambo,10440,Хрустальный,"триллеры, детективы"
1,rambo,13865,Девятаев,"драмы, военные, приключения"
2,rambo,15297,Клиника счастья,"драмы, мелодрамы"
3,rambo,9728,Гнев человеческий,"боевики, триллеры"
4,rambo,3734,Прабабушка легкого поведения,комедии
5,rambo,4151,Секреты семейной жизни,комедии
6,rambo,4880,Афера,комедии
7,rambo,11237,День города,комедии
8,rambo,2657,Подслушано,"драмы, триллеры"
9,rambo,7571,100% волк,"мультфильм, приключения, семейное, фэнтези, ко..."


In [54]:
# рекомендации для аватара madam_hihi

user_id = "madam_hihi"

recommended_items = get_recos(user_id=user_id,
                              model=model,
                              inter_matrix=inter_matrix,
                              user_to_id=DATASET._user_id_mapping,
                              id_to_item=id_item_map,
                              n_recommendations=N
)

madam_hihi_recos = pd.DataFrame({"user_id": user_id, "item_id": recommended_items}).merge(item_df[["item_id", "title", "genres"]])
madam_hihi_recos

,user_id,item_id,title,genres
0,madam_hihi,3734,Прабабушка легкого поведения,комедии
1,madam_hihi,15297,Клиника счастья,"драмы, мелодрамы"
2,madam_hihi,9728,Гнев человеческий,"боевики, триллеры"
3,madam_hihi,10440,Хрустальный,"триллеры, детективы"
4,madam_hihi,13865,Девятаев,"драмы, военные, приключения"
5,madam_hihi,4151,Секреты семейной жизни,комедии
6,madam_hihi,7571,100% волк,"мультфильм, приключения, семейное, фэнтези, ко..."
7,madam_hihi,16166,Зверополис,"приключения, мультфильм, детективы, комедии"
8,madam_hihi,4880,Афера,комедии
9,madam_hihi,11237,День города,комедии
